In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import musx
import rtmidi
from custom_utils import *

In [2]:
midiout = rtmidi.MidiOut()
outports = midiout.get_ports()
print("available ports:", outports)

available ports: ['Microsoft GS Wavetable Synth 0', 'musx2reaper 1']


In [3]:
midiout.open_port(outports.index('musx2reaper 1'))

In [83]:
def cc_init(queue):
    queue.out.addevent(musx.MidiEvent.control_change(0, 32, 0)) # Reset Chain Reverb mix to 0%
    queue.out.addevent(musx.MidiEvent.control_change(2, 32, 0)) # Reset volume mix to -inf dB
    queue.out.addevent(musx.MidiEvent.control_change(4, 32, 0))
    queue.out.addevent(musx.MidiEvent.control_change(4, 33, 88)) # Reset bottle-high panning to center
    queue.out.addevent(musx.MidiEvent.control_change(4, 34, 88)) # Reset bottle-high panning to center
    yield 0

In [74]:
seq = musx.MidiSeq()
queue = musx.Scheduler(out=seq)
queue.compose([[0, cc_init(queue)],
               [0, random_ascend(queue, chan=0, length=60, notes=musx.scale(60, 17, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
               [0, cc_distribution(queue, chan=0, ctrl=33, length=60, rate=0.25)],
               [0, random_ascend(queue, chan=1, length=60, notes=musx.scale(60, 17, 2, 2, 1, 2, 2, 2, 1), dur=1)],
               [0, cc_distribution(queue, chan=0, ctrl=34, length=60, rate=0.5)],
               [30, cc_linear(queue, chan=0, ctrl=32, length=30, start=0, end=0.25)]
               ])
seq.play(midiout, False)

Done!


In [11]:
def beads(queue):
    queue.compose([
       [0, distribution(queue, chan=0, length=40, notes=musx.scale(0, 7, 2, 2, 1, 2, 2, 2, 1), dur=1)],
       [8, distribution(queue, chan=0, length=32, notes=musx.scale(24, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
       [16, distribution(queue, chan=0, length=24, notes=musx.scale(36, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
       [24, distribution(queue, chan=0, length=16, notes=musx.scale(48, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
       [32, distribution(queue, chan=0, length=8, notes=musx.scale(60, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
       [40, distribution(queue, chan=0, length=28, notes=musx.scale(36, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
       [40, cc_linear(queue, chan=0, ctrl=40, length=16, start=0, end=32, low=0, high=127)],
       [56, distribution(queue, chan=0, length=16, notes=musx.scale(0, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
       [60, distribution(queue, chan=0, length=28, notes=musx.scale(24, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.25)],
       [64, distribution(queue, chan=0, length=24, notes=musx.scale(36, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
       [68, distribution(queue, chan=0, length=20, notes=musx.scale(48, 7, 2, 2, 1, 2, 2, 2, 1), dur=0.125)],
                   ])
    yield 0

In [7]:
def bead_to_blow(queue, *, chan1, chan2, length, notes, rhy, distribution=musx.uniran, lo=0, hi=1):
    start_time = queue.now
    while queue.now - start_time < length:
        selected = round(musx.rescale(distribution(), lo, hi, 0, len(notes) - 1))
        queue.out.addevent(musx.MidiNote(time=queue.now, dur=rhy, key=notes[selected], chan=chan1))
        queue.out.addevent(musx.MidiNote(time=queue.now, dur=rhy, key=notes[selected] + 60, chan=chan2))
        yield rhy

In [63]:
def transition1(queue):
    queue.compose([[0, bead_to_blow(queue, chan1=0, chan2=2, length=16, notes=musx.scale(24, 7, 2, 2, 1, 2, 2, 2, 1), rhy=0.5)],
                   [4, cc_linear(queue, chan=2, ctrl=20, length=12, start=0, end=24, low=0, high=127)],])
    yield 0 

In [88]:
def blown_bottles(queue):
    for iteration in range(36):
        jumble_gen = musx.jumble(musx.scale(48, 23, 2, 2, 1, 2, 2, 2, 1))
        notes = [next(jumble_gen) for _ in range(6)]
        for note in notes:
            events = round(musx.rescale(musx.midran(), 0, 1, 1, 16))
            rotation = round(musx.rescale(musx.uniran(), 0, 1, 0, 15))
            queue.compose([[iteration * 4, euclidean_rhythm(queue, chan=2, length=4, keynum=note, events=events, positions=16, cycletime=4, rotation=rotation)]])
    yield 0

In [91]:
seq = musx.MidiSeq()
queue = musx.Scheduler(out=seq)
# queue.compose([[0, cc_init(queue)],
#                [0, beads(queue)]])
queue.compose([[0, cc_init(queue)],
               [0, beads(queue)],
            #    [88, transition1(queue)],
               [88, blown_bottles(queue)],
               [100, cc_linear(queue, chan=2, ctrl=32, length=60, start=0, end=1)],
               [170, random_ascend(queue, chan=4, length=60, notes=musx.scale(84, 17, 2, 2, 1, 2, 2, 2, 1), dur=0.5)],
               [170, cc_distribution(queue, chan=4, ctrl=33, length=60, rate=0.25)],
               [170, random_ascend(queue, chan=5, length=60, notes=musx.scale(84, 17, 2, 2, 1, 2, 2, 2, 1), dur=1)],
               [170, cc_distribution(queue, chan=4, ctrl=34, length=60, rate=0.5)],
               [200, cc_linear(queue, chan=4, ctrl=32, length=30, start=0, end=0.25)]
               ])
# midiout.send_message(musx.control_change(2, 32, 12))
# queue.compose([[0, blown_bottles(queue)]])             
seq.play(midiout, False)

Done!


In [51]:
midiout = midi_restart(midiout, 'musx2reaper 1')

In [87]:
midiout.send_message(musx.control_change(4, 34, 40)) # used for MIDI learning the Supermassive parameter
# midiout.send_message(musx.control_change(0, 33, 64)) # used for MIDI learning bottle-high pan parameter
# midiout.send_message(musx.control_change(0, 34, 64)) # used for MIDI learning bottle-high pan parameter

In [ ]:
"""
Notes for myself:
- Multitrack is the best way to make sonic material (can pan sounds differently)
- "Chaos theory"
- Super high pitched bottles (108+) are digital like
"""

In [36]:
seq = musx.MidiSeq()
queue = musx.Scheduler(out=seq)
queue.compose([[0, cc_init(queue)],
               [0, distribution(queue, 0, 8, musx.scale(60, 7, 2, 2, 1, 2, 2, 2, 1), 0.5, musx.uniran, 0, 1)],
               [8, distribution(queue, 0, 8, musx.scale(60, 7, 2, 2, 1, 2, 2, 2, 1), 0.25, musx.uniran, 0, 1)],
               [8, distribution(queue, 0, 8, musx.scale(72, 7, 2, 2, 1, 2, 2, 2, 1), 0.25, musx.uniran, 0, 1)],
               [16, distribution(queue, 0, 8, musx.scale(48, 7, 2, 2, 1, 2, 2, 2, 1), 0.125, musx.uniran, 0, 1)],
               [16, distribution(queue, 0, 8, musx.scale(60, 7, 2, 2, 1, 2, 2, 2, 1), 0.125, musx.uniran, 0, 1)],
               [16, distribution(queue, 0, 8, musx.scale(72, 7, 2, 2, 1, 2, 2, 2, 1), 0.125, musx.uniran, 0, 1)],
               [16, distribution(queue, 0, 8, musx.scale(84, 7, 2, 2, 1, 2, 2, 2, 1), 0.125, musx.uniran, 0, 1)],
               [16, cc_linear(queue, 0, 32, length=16, start=0, end=0.25)],
               [24, distribution(queue, 0, 8, musx.scale(48, 7, 2, 2, 1, 2, 2, 2, 1), 0.0625, musx.uniran, 0, 1)],
               [24, distribution(queue, 0, 8, musx.scale(60, 7, 2, 2, 1, 2, 2, 2, 1), 0.0625, musx.uniran, 0, 1)],
               [24, distribution(queue, 0, 8, musx.scale(72, 7, 2, 2, 1, 2, 2, 2, 1), 0.0625, musx.uniran, 0, 1)],
               [24, distribution(queue, 0, 8, musx.scale(84, 7, 2, 2, 1, 2, 2, 2, 1), 0.0625, musx.uniran, 0, 1)],])
seq.play(midiout, False)

Done!
